
# Project Background


**GoodThought NGO** has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. This project, wishes to engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="100%" height="100%">

_Figure 1: ERD diagram (Main souce: DataCamp: https://app.datacamp.com/learn/projects/2190)_

# **Data Preview**

In [37]:
-- Truncated preview of donations table
SELECT *
FROM donations
LIMIT 10;

,donation_id,donor_id,amount,donation_date,assignment_id
0,1,2733,271.36,2021-08-21 00:00:00+00:00,4226
1,2,2608,251.49,2021-10-15 00:00:00+00:00,1323
2,3,1654,528.38,2020-03-03 00:00:00+00:00,4881
3,4,3265,730.36,2021-02-06 00:00:00+00:00,1237
4,5,4932,285.96,2022-03-05 00:00:00+00:00,1626
5,6,4860,189.50,2022-01-24 00:00:00+00:00,1409
6,7,1034,720.93,2021-09-21 00:00:00+00:00,4353
7,8,4374,938.31,2021-04-16 00:00:00+00:00,774
8,9,3469,370.64,2022-01-10 00:00:00+00:00,4810
9,10,706,488.46,2021-07-10 00:00:00+00:00,3190


**Donations table**

| **Column Name** | **Data Type** | **Description**                                                             |
| --------------- | ------------- | --------------------------------------------------------------------------- |
| `donation_id`   | integer       | Primary key; unique identifier for each donation                            |
| `donor_id`      | integer       | Foreign key to `donors.donor_id`; links donation to a specific donor        |
| `amount`        | decimal       | Monetary amount donated                                                     |
| `donation_date` | text          | Date of the donation (ideally should be a date type)                        |
| `assignment_id` | integer       | Foreign key to `assignments.assignment_id`; links donation to an assignment |


**Observation:** Although the status and created_at fields are explicitly defined in the ERD, they do not appear as filterable columns in the interface. Attempting to manually filter by these fields returns an error stating the columns do not exist.

In [8]:
-- Truncated preview of assignments table
SELECT * 
FROM assignments
LIMIT 10;

,assignment_id,assignment_name,start_date,end_date,budget,region,impact_score
0,1,Assignment_1,2021-10-17,2021-12-04,-32322.03,West,5.55
1,2,Assignment_2,2020-10-26,2020-11-28,57278.40,South,1.45
2,3,Assignment_3,2021-08-11,2022-03-17,40414.51,West,2.34
3,4,Assignment_4,2021-11-22,2022-05-17,31732.48,East,7.05
4,5,Assignment_5,2020-11-22,2021-07-10,13548.22,North,5.29
5,6,Assignment_6,2020-01-02,2021-01-17,46884.12,East,6.38
6,7,Assignment_7,2020-07-20,2020-11-18,15638.08,East,5.12
7,8,Assignment_8,2021-02-01,2021-03-10,83765.95,North,9.33
8,9,Assignment_9,2021-12-01,2021-07-12,94549.41,North,8.43
9,10,Assignment_10,2020-10-11,2020-12-23,7516.23,South,9.82


**Assignment table**

| **Column Name**   | **Data Type** | **Description**                                                |
| ----------------- | ------------- | -------------------------------------------------------------- |
| `assignment_id`   | integer       | Primary key; unique identifier for each assignment             |
| `assignment_name` | varchar       | Name/title of the assignment/project                           |
| `start_date`      | varchar       | Start date of the assignment (ideally should be a date type)   |
| `end_date`        | varchar       | End date of the assignment (ideally should be a date type)     |
| `budget`          | decimal       | Total planned financial allocation for the assignment          |
| `region`          | varchar       | Geographical region where the assignment is being executed     |
| `impact_score`    | decimal       | Score indicating the effectiveness or impact of the assignment |

In [2]:
-- Truncated preview of donors table
SELECT *
FROM donors
LIMIT 10;

,donor_id,donor_name,donor_type
0,1,Donor_1,Individual
1,2,Donor_2,Organization
2,3,Donor_3,Individual
3,4,Donor_4,Organization
4,5,Donor_5,Organization
5,6,Donor_6,Corporate
6,7,Donor_7,Individual
7,8,Donor_8,Corporate
8,9,Donor_9,Individual
9,10,Donor_10,Individual


**Donor table**

| **Column Name** | **Data Type** | **Description**                                  |
| --------------- | ------------- | ------------------------------------------------ |
| `donor_id`      | integer       | Primary key; unique identifier for each donor    |
| `donor_name`    | varchar       | Name of the donor  |
| `donor_type`    | varchar       | Type of donor (Individual, Corporate, Organization) |


## Observations in **assignment table**

There are a lot of negative values in the budget column, but most of negative values are almost as identical to the non negative budgets. Possible reasons are:

1. **Data Entry Error:**
Negative signs were accidentally added during data entry, especially if values are mirrored (e.g., 25000 and -25000).

2. **Refunds/Reversals:**
These could represent canceled or reversed budgets — if a project was terminated, and the budget was returned or not used.

3. **Accounting Adjustments:**
Sometimes budgets are logged as debits and credits — negative values might mean "allocated" while positives are "spent" or vice versa.

In [2]:
-- after detecting negative values during preliminary data preview in assignment table, this query seeks to validate and investigate all instances of negative values in the budget column.

SELECT 
	assignment_id, 
	assignment_name, 
	budget
FROM assignments
WHERE budget < 0
ORDER BY ABS(budget);

,assignment_id,assignment_name,budget
0,4230,Assignment_4230,-27.48
1,303,Assignment_303,-28.23
2,933,Assignment_933,-39.19
3,4562,Assignment_4562,-75.36
4,2127,Assignment_2127,-162.66
...,...,...,...
1725,4051,Assignment_4051,-80156.50
1726,2401,Assignment_2401,-83002.48
1727,3751,Assignment_3751,-83594.66
1728,301,Assignment_301,-85983.32


In [53]:
-- Cleaned budget for analysis
SELECT 
    assignment_id,
    assignment_name,
    region,
	ABS(budget) AS budget, -- removes any negative sign, without changing original data.
    impact_score
FROM assignments;

,assignment_id,assignment_name,region,budget,impact_score
0,1,Assignment_1,West,32322.03,5.55
1,2,Assignment_2,South,57278.40,1.45
2,3,Assignment_3,West,40414.51,2.34
3,4,Assignment_4,East,31732.48,7.05
4,5,Assignment_5,North,13548.22,5.29
...,...,...,...,...,...
4995,4996,Assignment_4996,West,17276.64,5.85
4996,4997,Assignment_4997,South,14681.98,4.68
4997,4998,Assignment_4998,West,20391.76,3.46
4998,4999,Assignment_4999,South,9803.08,9.55


# Analysis Section

**The analysis problems for Items B and C were provided by DataCamp, whereas the subsequent analyses were independently configured and conducted by the author.**

### **A. Regional Assignment Distribution Analysis**
Which regions received the most projects/assignments.
1. Rank regions by assignment volume
2. Calculate regional distribution percentages

In [6]:
-- A. regional_assignment_distribution_analysis
SELECT 
    region, 
    COUNT(*) AS total_assignments,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM assignments), 2) AS percentage 
FROM assignments
GROUP BY region
ORDER BY total_assignments DESC;

,region,total_assignments,percentage
0,South,1276,25.52
1,West,1254,25.08
2,East,1246,24.92
3,North,1224,24.48


**Observations:**

- South region leads with the highest project allocation at 25.25% of total assignments
- North region shows the lowest allocation at 24.48%
- Marginal difference of just 0.77 percentage points separates highest and lowest regions

##  B. Highest Donations Assignment per Region
Determining the top five assignments based on total value of donations, categorized by donor type. The target output should include four columns:**

1) assignment_name,
2) region,
3) rounded_total_donation_amount rounded to two decimal places, and
4) donor_type, sorted by rounded_total_donation_amount in descending order.

In [5]:
-- B. highest_donation_assignments
WITH donations_total AS (
	SELECT 
	    d.assignment_id,
	    don.donor_type,
	    ROUND(SUM(d.amount), 2) AS rounded_total_donation_amount
	FROM donations AS d
	INNER JOIN donors AS don
	  ON d.donor_id = don.donor_id
	GROUP BY  d.assignment_id, don.donor_type
)
	
SELECT 
	a.assignment_name,
	a.region,
	dt.rounded_total_donation_amount,
	dt.donor_type
FROM donations_total AS dt
INNER JOIN assignments AS a 
	ON dt.assignment_id = a.assignment_id
ORDER BY dt.rounded_total_donation_amount DESC
LIMIT 5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


**Observation:** The top five assignments in terms of total donation value are Assignments 3033, 300, 4114, 1765, and 265, originating exclusively from the East, West, and North regions, with no entries recorded from the South region. 

Organizations tend to give larger donations than individuals, with 3 of the top 5 being from organizations.

## C. Top Regional Impact Assignments
Identify the assignment with the highest impact score in each region, ensuring that each listed assignment has received at least one donation. The output should include four columns:**

1) assignment_name,
2) region,
3) impact_score, and
4) num_total_donations, sorted by region in ascending order.

To include only the highest-scoring assignment per region, avoiding duplicates within the same region.

In [5]:
-- C. top_regional_impact_assignments

WITH donation_per_assignments AS (
	
	SELECT 
	a.assignment_name,
	COUNT(d.amount) AS num_total_donations      
	FROM donations AS d
	INNER JOIN assignments AS a 
	    ON d.assignment_id = a.assignment_id
	GROUP BY a.assignment_name
),

region_scores AS (
	SELECT 
	   assignment_name,
	   region,
	   impact_score,
	   ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) AS rnk
	FROM assignments
)

SELECT 
      da.assignment_name,
      rs.region,
      rs.impact_score, 
      da.num_total_donations 
FROM donation_per_assignments AS da
INNER JOIN region_scores AS rs
     ON da.assignment_name = rs.assignment_name
WHERE rs.rnk = 1
ORDER BY rs.region ASC;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_3764,West,9.99,1


**Key Findings:**

- East and South regions have perfect impact scores (10.00)
- North and West are very close behind (9.99)
- The East region assignment received the most donations (2) among top performers

## D. Are the highest-impact assignments receiving enough funding?

After identifying the highest-impact assignments per region (as outlined in Item B), this section evaluates whether these initiatives are receiving adequate funding to maximize their effectiveness.

In [29]:
-- D. are the highest-impact assignments receiving enough funding

WITH donations_per_assignment AS (
	SELECT 
	a.assignment_name,
	a.region,
	a.impact_score,
	ABS(a.budget) AS cleaned_budget,
	ROUND(SUM(amount), 2) AS total_donations
	FROM assignments AS a 
	INNER JOIN donations AS d
		ON a.assignment_id = d.assignment_id
	GROUP BY a.assignment_name,a.region, a.impact_score, a.budget
),

assignment_scores AS (
	SELECT 
		dpa.assignment_name,
		dpa.region,
		dpa.impact_score,
		SUM(cleaned_budget) AS total_allocated_budget,
		ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) AS score_rnk
	FROM donations_per_assignment AS dpa
	GROUP BY dpa.assignment_name, dpa.region, dpa.impact_score
)

SELECT
	da.assignment_name,
	ac.region,
	ac.impact_score,
	ac.total_allocated_budget,
	da.total_donations,
	CASE 
        WHEN COALESCE(da.total_donations, 0) >= ac.total_allocated_budget THEN 'Sufficiently Funded'
        ELSE 'Underfunded'
    END AS funding_status
FROM donations_per_assignment AS da
INNER JOIN assignment_scores AS ac
	ON da.assignment_name = ac.assignment_name
WHERE ac.score_rnk = 1	
ORDER BY ac.impact_score DESC;

,assignment_name,region,impact_score,total_allocated_budget,total_donations,funding_status
0,Assignment_316,East,10.00,13194.92,530.81,Underfunded
1,Assignment_3547,South,10.00,69255.26,880.15,Underfunded
2,Assignment_3764,West,9.99,96886.51,716.72,Underfunded
3,Assignment_2253,North,9.99,45428.43,872.03,Underfunded


**Observation:** Despite their high impact scores, the top assignments in each region remain underfunded. The results indicate that all identified high-impact initiatives currently lack sufficient financial support.


## E. Top 5 Donors by Total Contributions
Recognize the most valuable contributors.

In [5]:
-- E. top-5-donors-by-total-contributions

SELECT 
	d.donor_name, 
	SUM(dn.amount) AS total_donated
FROM donors AS d
JOIN donations AS dn 
	ON d.donor_id = dn.donor_id
GROUP BY d.donor_name
ORDER BY total_donated DESC
LIMIT 5;

,donor_name,total_donated
0,Donor_1796,3714.09
1,Donor_368,3465.02
2,Donor_1988,3463.65
3,Donor_2767,3298.47
4,Donor_1839,3180.59


## F. Donor Funding Patterns by Assignment and Region

1. Which assignments receive the most funding from each donor type?
2. Which regions do these highly funded assignments belong to?

In [10]:
-- F. donor-funding-patterns-by-assignment-and-region

WITH donor_assignments AS (
    SELECT 
        dn.donor_type,
        a.assignment_name,
        a.region,
        SUM(ABS(ds.amount)) AS total_donation
    FROM donations ds
    JOIN donors dn 
		ON ds.donor_id = dn.donor_id
    JOIN assignments a 
		ON ds.assignment_id = a.assignment_id
    GROUP BY dn.donor_type, 
			 a.assignment_name, 
			 a.region
),

ranked_assignments AS (
    SELECT 
        donor_type,
        assignment_name,
        region,
        total_donation,
        ROW_NUMBER() OVER(PARTITION BY donor_type ORDER BY total_donation DESC) AS assignment_rank
    FROM donor_assignments
)

SELECT 
    donor_type,
    assignment_name AS top_assignment,
    region,
    total_donation
FROM ranked_assignments
WHERE assignment_rank = 1
ORDER BY total_donation DESC;

,donor_type,top_assignment,region,total_donation
0,Individual,Assignment_3033,East,3840.66
1,Organization,Assignment_300,West,3133.98
2,Corporate,Assignment_3324,North,2430.68


**Observation: Funding priorities vary significantly by donor type:**

- **Individuals** predominantly fund Assignment 3033 (East region)
- **Organizations** primarily support Assignment 300 (West region)
- **Corporations** direct most funding to Assignment 3324 (North region)


## G. Assignment Recommendations by Impact Classification

1. Categorize assignments as High, Medium, or Low Impact based on their impact scores.
2. Determine recommended actions to optimize support for each impact tier.

In [9]:
-- G. assignment-recommendations-by-impact-classification

WITH donor_patterns AS(
	SELECT
	    a.assignment_name,
		COUNT(dn.donor_name) OVER(PARTITION BY a.assignment_name ORDER BY a.assignment_name) AS total_donors,
		a.impact_score
	FROM donations ds
	JOIN donors dn 
		ON ds.donor_id = dn.donor_id
	JOIN assignments a 
		ON ds.assignment_id = a.assignment_id
	GROUP BY  
		a.assignment_name, 
		a.region,
		a.impact_score,
		dn.donor_name
),
assignment_categories AS (
	SELECT 
		assignment_name,
		total_donors,
		impact_score,
		    CASE 
        WHEN impact_score >= 5 THEN 'High Impact'
        WHEN impact_score >= 3 AND impact_score < 5 THEN 'Medium Impact'
        WHEN impact_score < 3  THEN 'Low Impact'
        WHEN total_donors = 0 THEN 'No Donors'
        ELSE 'Needs Review'
    END AS impact_category,
    CASE 
        WHEN impact_score >= 5 AND total_donors > 1 THEN 'Priority - Expand this assignments'
        WHEN impact_score >= 5 AND total_donors = 1 THEN 'Potential - Find similar donors'
        WHEN impact_score >= 3 THEN 'Maintain - Steady performance'
        WHEN impact_score < 3 AND total_donors > 0 THEN 'Improve - Needs strategy'
        ELSE 'Investigate - Special case'
    END AS recommended_action
	FROM donor_patterns 
)

SELECT 
	assignment_name,
	total_donors,
	impact_score,
	impact_category,
	recommended_action
FROM assignment_categories
GROUP BY assignment_name,
		 total_donors,
		 impact_score,
		 impact_category,
		 recommended_action
ORDER BY assignment_name ASC;

,assignment_name,total_donors,impact_score,impact_category,recommended_action
0,Assignment_1,1,5.55,High Impact,Potential - Find similar donors
1,Assignment_10,2,9.82,High Impact,Priority - Expand this assignments
2,Assignment_100,2,4.08,Medium Impact,Maintain - Steady performance
3,Assignment_1000,2,2.01,Low Impact,Improve - Needs strategy
4,Assignment_1001,1,8.53,High Impact,Potential - Find similar donors
...,...,...,...,...,...
3183,Assignment_989,1,2.72,Low Impact,Improve - Needs strategy
3184,Assignment_99,2,1.50,Low Impact,Improve - Needs strategy
3185,Assignment_992,1,6.54,High Impact,Potential - Find similar donors
3186,Assignment_994,1,7.48,High Impact,Potential - Find similar donors


## H. Annual Donation Analysis

1. How has donation behavior changed annualy?
2. In which years did the number of donations gathered significantly?
3. Which year recorded the highest donation?

In [18]:
-- H. annual-donation-analysis

WITH yearly_donations AS (
    SELECT
    EXTRACT(YEAR FROM donation_date) AS year,
    COUNT(*) AS gathered_donations,
    SUM(amount) AS total_donation_amount
    FROM donations 
    GROUP BY EXTRACT(YEAR FROM donation_date)
),

annual_difference AS (
    SELECT 
      year,
      gathered_donations,
      total_donation_amount,
      total_donation_amount - LAG(total_donation_amount) OVER(ORDER BY year) AS budget_difference,
      LAG(total_donation_amount) OVER(ORDER BY year) AS prev_year_amount
    FROM yearly_donations
)

SELECT 
    year,
    gathered_donations,
    total_donation_amount,
    budget_difference,
    CASE 
        WHEN prev_year_amount IS NULL THEN 'First year - No comparison'
        WHEN budget_difference > 0 THEN 'Budget Increased'
        WHEN budget_difference = 0 THEN 'Budget Steady'
        WHEN budget_difference < 0 THEN 'Budget Decreased'
        ELSE 'No budget difference'
    END AS status
FROM annual_difference


,year,gathered_donations,total_donation_amount,budget_difference,status
0,2020,1695,866914.68,NaN,First year - No comparison
1,2021,1651,819859.27,-47055.41,Budget Decreased
2,2022,1654,839805.34,19946.07,Budget Increased


**Observation:**

- **2020:** Record growth year with peak donor participation and highest total donations received.
- **2021:** Significant budget contraction observed, marked by decreased donor count and reduced donations.
- **2022:** Robust recovery achieved, surpassing 2021 performance metrics in both donor acquisition and total funding.

# CONCLUSION

The analysis of the GoodThought NGO's PostgreSQL database from 2010 to 2023 uncovers significant insights into its operational and financial landscape. One of the major data quality concerns observed was the presence of negative values in the **budget column** of the _assignments table_, likely due to data entry errors, accounting adjustments, or reversed allocations. This suggests a need for improved validation in financial data entry processes. The distribution of large donations revealed a regional imbalance, with the **East, West, and North regions** receiving the top-funded assignments, while the **South region had no representation among top donation recipients despite having the most number of assignments by 25.25%**. Moreover, organizations tend to contribute more substantial funds compared to individuals and corporations.

Despite having perfect or near-perfect impact scores across all regions, **high-impact assignments remain underfunded**, highlighting a disconnect between project potential and actual donor support. The type of donor significantly influences funding priorities, with individuals, organizations, and corporations each focusing on different regions and assignments. From a temporal perspective, 2020 marked a peak year in both donor engagement and total donations, while 2021 experienced a decline, followed by a strong recovery in 2022, indicating resilience and regained donor confidence.

These findings emphasize the need for better alignment between impact potential and funding allocation, enhanced data integrity measures, and targeted engagement strategies based on donor behavior and regional performance. Moving forward, strategic actions addressing these disparities can help maximize the NGO’s social impact and operational effectiveness.